# 卷积神经网络

- 图像数据的每个样本都由一个二维像素网格组成，每个像素可能是一个或者多个数值，取决于是黑白还是彩色图像
- 到目前为止，我们处理这类结构丰富的数据的方式仅仅通过将图像数据展平成一维向量而忽略了每个图像的空间结构信息，再将数据送入一个全连接的多层感知机中

<center><img src="../img/6_convolutional_neural_networks/pixels.png" width=60%></center>

<center><img src="../img/6_convolutional_neural_networks/rgbchannels.png" width=80%></center>

- 因为这些网络特征元素的顺序是不变的，因此最优的结果是利用先验知识，即利用相近像素之间的相互关联性，从图像数据中学习得到有效的模型

- **卷积神经网络**（convolutional neural network，CNN）是一类强大的、为处理图像数据而设计的神经网络
- 基于卷积神经网络架构的模型在计算机视觉领域中已经占主导地位，当今几乎所有的图像识别、目标检测或语义分割相关的学术竞赛和商业应用都以这种方法为基础

- 卷积神经网络需要的参数少于全连接架构的网络，而且卷积也很容易用GPU并行计算
- 因此卷积神经网络除了能够高效地采样从而获得精确的模型，还能够高效地计算

# 从全连接层到卷积

## 计算机视觉

- 计算机视觉（computer vision, CV）的任务：
    - 物体分类（classification）：图像中的物体属于哪个类别？
    - 物体辨认（identification）：图像中给定的物体是哪种类型的？
    - 物体确认（verification）：这个物体在图像里吗？
    - 物体检测（detection）：物体在图像中的哪个位置？
    - 物体特征检测（landmark detection）：图像中物体的关键点是什么？
    - 物体分割（segmentation）：图像中的哪些像素属于指定的物体？
    - 物体认识（recognition）：这张图像中有哪些物体，它们在哪里？

<center><img src="../img/6_convolutional_neural_networks/computerVisionTasks.png" width=80%></center>

## 全连接层的局限

- 之前讨论的多层感知机十分适合处理表格数据，其中行对应样本，列对应特征
- 对于表格数据，寻找的模式可能涉及特征之间的交互，但是不能预先假设任何与特征交互相关的先验结构
- 此时，多层感知机可能是最好的选择

- **局限1：对于高维感知数据，这种缺少结构的网络可能会变得不实用**

- 假设有一个足够充分的照片数据集，数据集中是拥有标注的照片，每张照片具有百万级像素，这意味着网络的每次输入都有一百万个维度
- 即使将隐藏层维度降低到1000，这个全连接层也将有$10^6 \times 10^3 = 10^9$个参数，想要训练这个模型将不可实现，因为需要有大量的GPU、分布式优化训练的经验和超乎常人的耐心

- **局限2：缺乏空间结构信息**

<center><img src="../img/6_convolutional_neural_networks/limitationsMLP.png" width=80%></center>

- 卷积神经网络是机器学习利用自然图像中一些已知结构的创造性方法

## 不变性

- 想从一张图片中找到某个物体，合理的假设是：无论哪种方法找到这个物体，都应该和物体的位置无关

- 卷积神经网络正是将**空间不变性**（spatial invariance）的概念系统化，从而基于这个模型使用较少的参数来学习有用的表示

- 空间不变性具体包括：
    - **平移不变性**（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应
    - **局部性**（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系。最终，可以聚合这些局部特征，以在整个图像级别进行预测

- 平移不变性

<center><img src="../img/6_convolutional_neural_networks/translationInvariance.png" width=80%></center>

- 局部性

<center><img src="../img/6_convolutional_neural_networks/locality.png" width=80%></center>

天空周围的像素仍然是天空，向日葵附近的像素更有可能是向日葵

## 多层感知机对图像的表示

- 多层感知机的输入是二维图像$\mathbf{X}$，其隐藏表示$\mathbf{H}$是一个矩阵，在代码中表示为二维张量。其中$\mathbf{X}$和$\mathbf{H}$具有相同的形状。

- 使用$[\mathbf{X}]_{i, j}$和$[\mathbf{H}]_{i, j}$分别表示输入图像和隐藏表示中位置（$i$,$j$）处的像素。

<center><img src="../img/6_convolutional_neural_networks/imgMlp4Dim.svg" width=60%></center>

- 为了使每个隐藏神经元都能接收到每个输入像素的信息，将参数从权重矩阵替换为四阶权重张量$\mathsf{W}$。假设$\mathbf{U}$包含偏置参数，可以将全连接层形式化地表示为

$$\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &=  [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.\end{aligned}
\label{eq:fullConnectRelative}$$

- 式\eqref{eq:fullConnectRelative}中，从$\mathsf{W}$到$\mathsf{V}$的转换只是形式上的转换，因为在这两个四阶张量的元素之间存在一一对应的关系
    - 从绝对位置变换为相对（$i$, $j$）的相对位置

- 索引$a$和$b$通过在正偏移和负偏移之间移动覆盖了整个图像
- 对于隐藏表示中任意给定位置（$i$,$j$）处的像素值$[\mathbf{H}]_{i, j}$，可以通过在$X$中以$(i, j)$为中心对像素进行加权求和得到，加权使用的权重为$[\mathsf{V}]_{i, j, a, b}$

### 平移不变性

- 意味着检测对象在输入$\mathbf{X}$中的平移，应该仅导致隐藏表示$\mathbf{H}$中的平移。也就是说，$\mathsf{V}$和$\mathbf{U}$实际上不依赖于$(i, j)$的值，并且$\mathbf{U}$是一个常数，比如$u$。因此，可以简化$\mathbf{H}$定义为：

$$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b} [\mathbf{X}]_{i+a, j+b}
\label{eq:convolution}$$

- 式\eqref{eq:convolution}表达的就是**卷积**（convolution），使用系数$[\mathbf{V}]_{a, b}$对位置$(i, j)$附近的像素$(i+a, j+b)$进行加权得到$[\mathbf{H}]_{i, j}$

- $[\mathbf{V}]_{a, b}$的系数比$[\mathsf{V}]_{i, j, a, b}$少很多，因为前者不再依赖于图像中的位置

### 局部性

- 为了收集用来训练参数$[\mathbf{H}]_{i, j}$的相关信息，不应偏离到距$(i, j)$很远的地方
- 这意味着在$|a|> \Delta$或$|b| > \Delta$的范围之外，可以设置$[\mathbf{V}]_{a, b} = 0$

- 因此，可以将$[\mathbf{H}]_{i, j}$重写为

$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}
\label{eq:convolutionLayer}$$

- 式\eqref{eq:convolutionLayer}是一个**卷积层**（convolutional layer），而卷积神经网络是包含卷积层的一类特殊的神经网络

- 在深度学习研究中，$\mathbf{V}$被称为**卷积核**（convolution kernel）或者**滤波器**（filter），亦或简单地称之为该卷积层的**权重**，通常该权重是可学习的参数

- $2\times2$卷积层示例

<center><img src="../img/6_convolutional_neural_networks/2by2convolution.png" width=60%></center>


- 当图像处理的局部区域很小时，卷积神经网络与多层感知机的训练有巨大差异
    - 多层感知机可能需要数十亿个参数来表示网络中的一层，而现在卷积神经网络通常只需要几百个参数，而且不需要改变输入或隐藏表示的维数

- 参数大幅减少的代价是，我们的特征现在是平移不变的，并且当确定每个隐藏活性值时，每一层只包含局部的信息
- 当图像不满足平移不变时，我们的模型可能难以拟合训练数据

## 卷积

- 在数学中，两个函数（比如$f, g: \mathbb{R}^d \to \mathbb{R}$）之间的“卷积”被定义为

$$(f * g)(\mathbf{t}) = \int f(\mathbf{\tau}) g(\mathbf{t}-\mathbf{\tau}) d\mathbf{\tau}.$$

- 卷积是当把一个函数沿y轴“翻转”并移位$\mathbf{t}$时，测量$f(\mathbf{\tau})$围成的面积，且以$g(\mathbf{t}-\mathbf{\tau})$为权重

<center><img src="../img/6_convolutional_neural_networks/Convolution3.svg" width=100%></center>

- 当为离散对象时，积分就变成求和。例如，对于由索引为$\mathbb{Z}$的、平方可和的、无限维向量集合中抽取的向量，得到以下定义：

$$(f * g)(i) = \sum_a f(a) g(i-a)$$

- 对于二维张量，则为$f$的索引$(a, b)$和$g$的索引$(i-a, j-b)$上的对应加和：

$$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i-a, j-b)
\label{eq:convolution2D}$$

- 卷积层根据滤波器$\mathbf{V}$选取给定大小的窗口，并加权处理图片
- 目标是学习一个模型，以便探测出在图像最可能出现的地方

### 通道

- 然而上述方法忽略了图像一般包含三个通道/三种原色（红色、绿色和蓝色）

- 图像不是二维张量，而是一个由高度、宽度和颜色组成的三维张量，比如包含$1024 \times 1024 \times 3$个像素
- 前两个轴与像素的空间位置有关，而第三个轴可以看作每个像素的多维表示

<div class="row">
    <div class="col-md-6">
        <img src="../img/6_convolutional_neural_networks/coloimage1.jpg" width=100%>
    </div>
    <div class="col-md-6">
        <img src="../img/6_convolutional_neural_networks/coloimage2.jpg" width=100%>
    </div>
</div>

- 因此，将$\mathsf{X}$索引为$[\mathsf{X}]_{i, j, k}$
- 由此卷积相应地调整为$[\mathsf{V}]_{a,b,c}$，而不是$[\mathbf{V}]_{a,b}$

- 由于输入图像是三维的，隐藏表示$\mathsf{H}$也最好采用三维张量
- 对于每一个空间位置，采用一组而不是一个隐藏表示。这样一组隐藏表示可以想象成一些互相堆叠的二维网格

- 因此，可以把隐藏表示想象为一系列具有二维张量的**通道**（channel）

- 因为每个通道都向后续层提供一组空间化的学习特征，因此，这些通道有时也被称为**特征映射**（feature maps）
    - 直观上可以想象在靠近输入的底层，一些通道专门识别边缘，而一些通道专门识别纹理

- 为了支持输入$\mathsf{X}$和隐藏表示$\mathsf{H}$中的多个通道，可以在$\mathsf{V}$中添加第四个坐标，即$[\mathsf{V}]_{a, b, c, d}$

- 综上所述，

$$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c}
\label{eq:convLayerChannels}$$

其中隐藏表示$\mathsf{H}$中的索引$d$表示输出通道，而随后的输出将继续以三维张量$\mathsf{H}$作为输入进入下一个卷积层。所以，式 \eqref{eq:convLayerChannels}可以定义具有多个通道的卷积层，而其中$\mathsf{V}$是该卷积层的权重

### 卷积层

- 考虑一个$5\times5\times3$的图像，一个卷积层有4个卷积核
- 卷积层的输出的形状为$4\times4\times4$

<center><img src="../img/6_convolutional_neural_networks/convolutionLayer.svg" width=100%></center>